# 06. Target encoding
In previous chapter, we've seen how to engineer numerical features. In this chapter, we will learn **target encoding**, which is instead meant for categorical features. It's a method of encoding categories as numbers, like **one-hot** or **label encoding**, with the difference that it also uses the target to create the encoding. This makes it what we call a **supervised feature engineering technique**.



In [1]:
import pandas as pd

In [2]:
data_path="../data/autos.csv"

In [3]:
autos=pd.read_csv(data_path)

In [4]:
autos.head()

,symboling,make,fuel_type,aspiration,num_of_doors,body_style,drive_wheels,engine_location,wheel_base,length,...,engine_size,fuel_system,bore,stroke,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price
0,3,alfa-romero,gas,std,2,convertible,rwd,front,88.6,168.8,...,130,mpfi,3.47,2.68,9,111,5000,21,27,13495
1,3,alfa-romero,gas,std,2,convertible,rwd,front,88.6,168.8,...,130,mpfi,3.47,2.68,9,111,5000,21,27,16500
2,1,alfa-romero,gas,std,2,hatchback,rwd,front,94.5,171.2,...,152,mpfi,2.68,3.47,9,154,5000,19,26,16500
3,2,audi,gas,std,4,sedan,fwd,front,99.8,176.6,...,109,mpfi,3.19,3.40,10,102,5500,24,30,13950
4,2,audi,gas,std,4,sedan,4wd,front,99.4,176.6,...,136,mpfi,3.19,3.40,8,115,5500,18,22,17450


## 6.1 Target Encoding

A target encoding is any kind of **encoding that replaces a feature's categories with some number derived from the target**.

A simple and effective version is to apply a group aggregation from chapter 3, like the mean. Using the Automobiles dataset, this computes the average price of each vehicle's make:

In [5]:
autos["make_encoded"] = autos.groupby("make")["price"].transform("mean")

autos[["make", "price", "make_encoded"]].head(10)

,make,price,make_encoded
0,alfa-romero,13495,15498.333333
1,alfa-romero,16500,15498.333333
2,alfa-romero,16500,15498.333333
3,audi,13950,17859.166667
4,audi,17450,17859.166667
5,audi,15250,17859.166667
6,audi,17710,17859.166667
7,audi,18920,17859.166667
8,audi,23875,17859.166667
9,bmw,16430,26118.750000


This kind of target encoding is sometimes called a **mean encoding**. Applied to a binary target, it's also called **bin counting**. (Other names you might come across include: `likelihood encoding, impact encoding, and leave-one-out encoding`.)

## 6.2 Smoothing

An encoding like this presents a couple of problems, however.

1. **unknown categories**: Target encodings create a special risk of overfitting, which means they need to be trained on an independent "encoding" split. When you join the encoding to future splits, Pandas will fill in missing values for any categories not present in the encoding split. These missing values you would have to impute somehow.

2. **rare categories**: When a category only occurs a few times in the dataset, any statistics calculated on its group are unlikely to be very accurate. In the Automobiles dataset, the `mercurcy` make only occurs once. The "mean" price we calculated is just the price of that one vehicle, which might not be very representative of any Mercuries we might see in the future. Target encoding rare categories can make overfitting more likely.

A solution to these problems is to add **smoothing**. The idea is to blend the `in-category average` with the `overall average`. Rare categories get less weight on their category average, while missing categories just get the overall average.

In pseudocode:

```text
encoding = weight * in_category + (1 - weight) * overall
```

where weight is a value between 0 and 1 calculated from the category frequency.

An easy way to determine the value for weight is to compute an **m-estimate**:

```text
weight = n / (n + m)

```
where `n` is the total number of times that category occurs in the data. The parameter `m` determines the "smoothing factor". Larger values of m put more weight on the overall estimate.

![06.TE_weight_calc.png](../img/06.TE_weight_calc.png)

In the Automobiles dataset there are three cars with the make chevrolet. If you chose m=2.0, then the chevrolet category would be encoded with 60% of the average Chevrolet price plus 40% of the overall average price.

```text
chevrolet = 0.6 * 6000.00 + 0.4 * 13285.03
```

### Tips on m value

When choosing a value for m, consider how noisy you expect the categories to be. Does the price of a vehicle vary a great deal within each make? Would you need a lot of data to get good estimates? If so, it could be better to choose a larger value for m; if the average price for each make were relatively stable, a smaller value could be okay.


## 6.3  Use Cases for Target Encoding

Target encoding is great for:
1. **High-cardinality features**: A feature with a large number of categories can be troublesome to encode: a one-hot encoding would generate too many features and alternatives, like a label encoding, might not be appropriate for that feature. A target encoding derives numbers for the categories using the feature's most important property: its relationship with the target.
2. **Domain-motivated features**: From prior experience, you might suspect that a categorical feature should be important even if it scored poorly with a feature metric. A target encoding can help reveal a feature's true informativeness.